In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: relu(x)
def acti(x):
    return torch.relu(x)

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()  
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()  
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) - 0)**2).cuda() 
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 1.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Neumann_relu.npy", error_save)
np.save("DRM_relative_error_4D_Neumann_relu.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(65.1008, device='cuda:1')
current relative error is:  tensor(2.5382, device='cuda:1')
current epoch is:  50
current loss is:  tensor(0.1658, device='cuda:1')
current relative error is:  tensor(1.0500, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-7.6287, device='cuda:1')
current relative error is:  tensor(0.7501, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-10.3919, device='cuda:1')
current relative error is:  tensor(0.5791, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-14.1124, device='cuda:1')
current relative error is:  tensor(0.4554, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-14.9739, device='cuda:1')
current relative error is:  tensor(0.3825, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-13.8525, device='cuda:1')
current relative error is:  tensor(0.3654, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-13.8223, devi

current epoch is:  3050
current loss is:  tensor(-17.3540, device='cuda:1')
current relative error is:  tensor(0.2095, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-17.3552, device='cuda:1')
current relative error is:  tensor(0.2227, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-15.8855, device='cuda:1')
current relative error is:  tensor(0.2205, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-16.5429, device='cuda:1')
current relative error is:  tensor(0.2070, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-16.3557, device='cuda:1')
current relative error is:  tensor(0.2177, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-16.6174, device='cuda:1')
current relative error is:  tensor(0.2066, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-15.9015, device='cuda:1')
current relative error is:  tensor(0.2037, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-16.8327, device='cuda:1')
current relative error is:  tensor(0.1795, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-16.7184, device='cuda:1')
current relative error is:  tensor(0.1874, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-18.1914, device='cuda:1')
current relative error is:  tensor(0.1886, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-16.6664, device='cuda:1')
current relative error is:  tensor(0.2014, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-17.4513, device='cuda:1')
current relative error is:  tensor(0.1944, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-16.7276, device='cuda:1')
current relative error is:  tensor(0.1893, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-16.7904, device='cuda:1')
current relative error is:  tensor(0.1768, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-18.1566, device='cuda:1')
current relative error is:  tensor(0.1771, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-15.7868, device='cuda:1')
current relative error is:  tensor(0.1997, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-18.3402, device='cuda:1')
current relative error is:  tensor(0.1751, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-16.4778, device='cuda:1')
current relative error is:  tensor(0.1797, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-17.1113, device='cuda:1')
current relative error is:  tensor(0.1712, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-18.1381, device='cuda:1')
current relative error is:  tensor(0.1699, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-16.9489, device='cuda:1')
current relative error is:  tensor(0.1763, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-17.6425, device='cuda:1')
current relative error is:  tensor(0.1455, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-18.7366, device='cuda:1')
current relative error is:  tensor(0.1299, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-19.0924, device='cuda:1')
current relative error is:  tensor(0.1443, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-16.6517, device='cuda:1')
current relative error is:  tensor(0.1275, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-19.0474, device='cuda:1')
current relative error is:  tensor(0.1300, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-17.2145, device='cuda:1')
current relative error is:  tensor(0.1264, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-17.2694, device='cuda:1')
current relative error is:  tensor(0.1359, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-20.1029, device='cuda:1')
current relative error is:  tensor(0.1349, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.3635, device='cuda:1')
current relative error is:  tensor(0.1013, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-18.2754, device='cuda:1')
current relative error is:  tensor(0.1064, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-19.2666, device='cuda:1')
current relative error is:  tensor(0.1109, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-19.4556, device='cuda:1')
current relative error is:  tensor(0.0939, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-17.9606, device='cuda:1')
current relative error is:  tensor(0.1365, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-18.8333, device='cuda:1')
current relative error is:  tensor(0.0831, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-19.4980, device='cuda:1')
current relative error is:  tensor(0.0725, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-19.4665, device='cuda:1')
current relative error is:  tensor(0.0745, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-19.2695, device='cuda:1')
current relative error is:  tensor(0.0746, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-19.2972, device='cuda:1')
current relative error is:  tensor(0.0752, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-17.3033, device='cuda:1')
current relative error is:  tensor(0.0743, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-18.9320, device='cuda:1')
current relative error is:  tensor(0.0867, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-18.7842, device='cuda:1')
current relative error is:  tensor(0.0857, device='cuda:1')
current epoch is:  18500
current loss is: